# Common Analysis for NBR Measurements

In [61]:
import Labber
import numpy as np
import os
import matplotlib.pyplot as plt
from fitTools.Resonator import Resonator
import logging
from resonator import reflection, see
import pandas as pd

## Powersweep fits

Specify the path to the hdfl5 file.

In [ ]:
fpath = r"G:\Shared drives\Labber\Data\2025\04\Data_0403\NBRL1A_no_rad_powerdweep_69mK_twpa_Off.hdf5"

In [ ]:
plt.rcParams.update({'font.size':14})
path,fname = os.path.split(fpath)
path += r'\\'
figpath = 'figures\\'+fname[:-4]+'\\'
if not os.path.exists(path+'figures\\'):
    os.mkdir(path+'figures\\')
if not os.path.exists(path+figpath):
    os.mkdir(path+figpath)
lf = Labber.LogFile(path + fname)

logFileName = path + f"profile_info_{fname[:-4]}log"
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger()
logger.addHandler(logging.FileHandler(logFileName, 'a'))
print = logger.info

nEntries = lf.getNumberOfEntries()
power = np.squeeze(np.round(lf.getData(name=lf.getStepChannels()[0]["name"]),decimals=2))

In [22]:
def adjust_range(f, s21, f_min, f_max):
    mask = (f >= f_min) & (f <= f_max)
    return f[mask], s21[mask]

def process_powersweep_fits(lf, nEntries, power, pdf_name, fmin, fmax):
    """
    Process fits for the given Labber log file and save the results to a PDF.
    
    Parameters:
    lf (Labber.LogFile): The Labber log file object.
    nEntries (int): The number of entries in the log file.
    power (list): The list of power values for each entry.
    pdf_name (str): The name of the PDF file to save the plots.
    fmin (float): The minimum frequency for the fit range.
    fmax (float): The maximum frequency for the fit range.
    
    """
    from matplotlib.backends.backend_pdf import PdfPages
    fits = {'f': [], 'Qint': [], 'Qext': [], 'power': [], 'Qtot': [], 'f_error': []}
    fit_objects = []

    # Create a PdfPages object to save all plots in one PDF
    with PdfPages(pdf_name) as pdf:
        for n in range(nEntries):
            (frequency, S21) = lf.getTraceXY(entry=n)
            frequency, S21 = adjust_range(frequency, S21, fmin, fmax)
            r = reflection.LinearReflectionFitter(frequency, S21)
            fit_objects.append(r)
            # Generate plots
            fig, (ax_magnitude, ax_phase, ax_complex) = see.triptych(
                resonator=r, plot_initial=False, frequency_scale=1e-9,
                figure_settings={'figsize': (16, 8), 'dpi': 300}
            )
            # Append power to the plot title
            fig.suptitle(f'DA Attenuation: {power[n]} dBm')
            ax_complex.legend()
            # Save the current figure to the PDF
            pdf.savefig(fig)
            plt.close(fig)
            # Append the fit results to the dictionary
            fits['f'].append(r.resonance_frequency)
            fits['Qtot'].append(r.Q_t)
            fits['Qint'].append(r.Q_i)
            fits['Qext'].append(r.Q_c)
            fits['power'].append(power[n])
            fits['f_error'].append(r.f_r_error)

    return fits, fit_objects

Specify the cut-off frequencies for a more accurate fit.

In [30]:
fmin, fmax=   5.7707e9, 5.7911e9

In [ ]:
fits, fit_objects = process_powersweep_fits(lf, nEntries, power, path + figpath + 'fits.pdf', fmin, fmax)

In [ ]:
# save the fit results in a csv file
powersweep_df = pd.DataFrame(fits)
powersweep_df.to_csv(path+figpath+'fit_results.csv', index=False)

In [41]:
# Create a single figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# Plot the fit results (frequency vs power with error) on the first subplot
ax1.errorbar(powersweep_df['power'], powersweep_df['f'], yerr=powersweep_df['f_error'], fmt='o-', label='Frequency with error')
ax1.set_xlabel('DA Attenuation (dBm)')
ax1.set_ylabel('Frequency (GHz)')
ax1.set_title('Frequency vs Power')
ax1.legend()
ax1.grid()

# Plot the Q factors on the second subplot
ax2.plot(powersweep_df['power'], powersweep_df['Qint'], 'o-', label='Qint')
ax2.plot(powersweep_df['power'], powersweep_df['Qext'], 'o-', label='Qext')
ax2.plot(powersweep_df['power'], powersweep_df['Qtot'], 'o-', label='Qtot')
ax2.set_xlabel('DA Attenuation (dBm)')
ax2.set_ylabel('Quality Factor')
ax2.set_title('Quality Factors vs Power')
ax2.legend()
ax2.grid()

# Adjust layout and save the figure as a PDF
plt.tight_layout()
pdf_path = path + figpath + 'combined_plots.pdf'
fig.savefig(pdf_path)
plt.close(fig)

## Fluxsweep fits

Specify the path to the hdf5 file.

In [62]:
fpath = r"G:\Shared drives\Labber\Data\2025\04\Data_0402\NBRL1A_70mK_flux_sweep.hdf5"

In [64]:
plt.rcParams.update({'font.size':14})


path,fname = os.path.split(fpath)
path += r'\\'
figpath = 'figures\\'+fname[:-4]+'\\'
if not os.path.exists(path+'figures\\'):
    os.mkdir(path+'figures\\')
if not os.path.exists(path+figpath):
    os.mkdir(path+figpath)
lf = Labber.LogFile(path + fname)

logFileName = path + f"profile_info_{fname[:-4]}log"
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger()
logger.addHandler(logging.FileHandler(logFileName, 'a'))
print = logger.info

nEntries = lf.getNumberOfEntries()
voltage = np.squeeze(np.round(lf.getData(name=lf.getStepChannels()[0]["name"]), decimals=6)) * 1e3 # mV


In [65]:
def process_fluxsweep_fits(lf, nEntries, voltage, pdf_name, span):
    from matplotlib.backends.backend_pdf import PdfPages
    fits = {'f': [], 'Qint': [], 'Qext': [], 'voltage': [], 'Qtot': [], 'f_error': []}
    fit_objects = []

    # Create a PdfPages object to save all plots in one PDF
    with PdfPages(pdf_name) as pdf:
        for n in range(nEntries):
            (frequency, S21) = lf.getTraceXY(entry=n)
            # Calculate the log magnitude of S21
            S21_log_mag = 20 * np.log10(np.abs(S21))
            # Find the index of the dip in the log magnitude
            dip_index = np.argmin(S21_log_mag)
            # Determine the center frequency
            center_frequency = frequency[dip_index]
            # Set fmin and fmax based on the span
            fmin = center_frequency - span / 2
            fmax = center_frequency + span / 2
            # Adjust the frequency and S21 range
            frequency, S21 = adjust_range(frequency, S21, fmin, fmax)
            r = reflection.LinearReflectionFitter(frequency, S21)
            fit_objects.append(r)
            # Generate plots
            fig, (ax_magnitude, ax_phase, ax_complex) = see.triptych(
                resonator=r, plot_initial=False, frequency_scale=1e-9,
                figure_settings={'figsize': (16, 8), 'dpi': 300}
            )
            # Append voltage to the plot title
            fig.suptitle(f'Voltage: {voltage[n]} mV')
            ax_complex.legend()
            # Save the current figure to the PDF
            pdf.savefig(fig)
            plt.close(fig)
            # Append the fit results to the dictionary
            fits['f'].append(r.resonance_frequency)
            fits['Qtot'].append(r.Q_t)
            fits['Qint'].append(r.Q_i)
            fits['Qext'].append(r.Q_c)
            fits['voltage'].append(voltage[n])
            fits['f_error'].append(r.f_r_error)

    return fits, fit_objects


In [66]:
fits, fit_objects = process_fluxsweep_fits(lf, nEntries, voltage, path + figpath + 'fits.pdf', span=20e6)

In [67]:
fluxsweep_df = pd.DataFrame(fits)
fluxsweep_df.to_csv(path+figpath+'fit_results.csv', index=False)

In [60]:
# plot the frequency and Q factors as a function of voltage
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# Plot the fit results (frequency vs voltage with error) on the first subplot
ax1.plot(fluxsweep_df['voltage'], fluxsweep_df['f'], 'o-', label='Frequency')
ax1.set_xlabel('Voltage (mV)')
ax1.set_ylabel('Frequency (GHz)')
ax1.set_title('Frequency vs Voltage')
ax1.legend()
ax1.grid()

# Plot the Q factors on the second subplot
ax2.plot(fluxsweep_df['voltage'], fluxsweep_df['Qint'], 'o-', label='Qint')
ax2.plot(fluxsweep_df['voltage'], fluxsweep_df['Qext'], 'o-', label='Qext')
ax2.plot(fluxsweep_df['voltage'], fluxsweep_df['Qtot'], 'o-', label='Qtot')
ax2.set_xlabel('Voltage (mV)')
ax2.set_ylabel('Quality Factor')
ax2.set_title('Quality Factors vs Voltage')
ax2.legend()
ax2.grid()

# Adjust layout and save the figure as a PDF
plt.tight_layout()
pdf_path = path + figpath + 'combined_plots.pdf'
fig.savefig(pdf_path)
plt.close(fig)
